In [441]:
import numpy as np
import pandas as pd
import pulp
from pulp import LpInteger

In [489]:

distance = np.loadtxt("./" +'distance.csv', delimiter=',', dtype='int64')

In [490]:

num = len(distance[0])
distance


array([[  0,  35,  78, ...,  62,  16,  73],
       [ 35,   0,  60, ...,  80,  19,  39],
       [ 78,  60,   0, ...,  72,  65,  48],
       ...,
       [ 62,  80,  72, ...,   0,  66, 102],
       [ 16,  19,  65, ...,  66,   0,  57],
       [ 73,  39,  48, ..., 102,  57,   0]])

In [491]:
num

32

In [492]:
problem = pulp.LpProblem( "TSP", pulp.LpMinimize )
problem

TSP:
MINIMIZE
None
VARIABLES

In [493]:
x = np.array([ [ pulp.LpVariable( 'x_{}_{}'.format( i, j ), cat="Binary" ) \
        if i != j else None for j in range(len(distance[0])) ] \
        for i in range(len(distance[0])) ])
y = np.array([ [ pulp.LpVariable( 'y_{}_{}'.format( i, j ), cat=LpInteger, lowBound=0) \
        if i != j else None for j in range(len(distance[0])) ] \
        for i in range(len(distance[0])) ])

In [494]:
y

array([[None, y_0_1, y_0_2, ..., y_0_29, y_0_30, y_0_31],
       [y_1_0, None, y_1_2, ..., y_1_29, y_1_30, y_1_31],
       [y_2_0, y_2_1, None, ..., y_2_29, y_2_30, y_2_31],
       ...,
       [y_29_0, y_29_1, y_29_2, ..., None, y_29_30, y_29_31],
       [y_30_0, y_30_1, y_30_2, ..., y_30_29, None, y_30_31],
       [y_31_0, y_31_1, y_31_2, ..., y_31_29, y_31_30, None]],
      dtype=object)

In [495]:
# 目的関数
problem += pulp.lpSum( distance[i][j] * x[i][j] for i in range(num) \
                        for j in range(num) if i != j )


In [496]:
for i in range(num):
    problem += pulp.lpSum( x[i][j] for j in range(num)) == 1


In [497]:
### \sum_{j \in V\{i}} x_ij = 1 for all i \in V\{0}
for i in range(num):
    problem += pulp.lpSum( x[h][i] for h in range(num)) == 1

In [498]:
# flow formation 制約
for i in range(num):
    for j in range(num):
        if i == j:
            None
        else:
            problem += y[i][j] <= (x[i][j] * (num-1))

In [499]:
for i in range(1,len(distance[0])):
    problem += pulp.lpSum( y[i][j] for j in range(len(distance[0])) if i != j ) - pulp.lpSum( y[h][i] for h in range(len(distance[0])) if i != h ) == 1

In [500]:
problem += pulp.lpSum( y[0][j] for j in range(len(distance[0])) if 0 != j ) - pulp.lpSum( y[h][0] for h in range(len(distance[0])) if 0 != h ) == -(num - 1)

In [501]:
problem += pulp.lpSum(y[0][j] for j in range(1,num)) == 0

In [502]:
import time

In [507]:
# solve
# threads = 8
# timeLimit = 60
# t1 = time.clock()
result = problem.solve()
# t2 = time.clock()
print("objective value = {}".format(pulp.value(problem.objective)))
# print(t2-t1)

objective value = 466.0


In [508]:
problem.status

1

In [505]:
print("objective value = {}".format(pulp.value(problem.objective)))

objective value = 466.0


In [506]:
for i in range(num):
    for j in range(num):
        if i !=j:
            if pulp.value(y[i][j]) != 0:
                print(y[i][j],pulp.value(y[i][j]))

y_1_7 5.0
y_2_3 13.0
y_3_23 14.0
y_4_11 17.0
y_5_20 27.0
y_6_2 12.0
y_7_13 6.0
y_8_18 19.0
y_9_22 21.0
y_10_25 25.0
y_11_8 18.0
y_12_1 4.0
y_13_21 7.0
y_14_0 31.0
y_15_29 23.0
y_16_12 3.0
y_17_6 11.0
y_18_9 20.0
y_19_17 10.0
y_20_27 28.0
y_21_31 8.0
y_22_15 22.0
y_23_28 15.0
y_24_14 30.0
y_25_5 26.0
y_26_16 2.0
y_27_24 29.0
y_28_4 16.0
y_29_10 24.0
y_30_26 1.0
y_31_19 9.0
